In [45]:
import requests
import pandas as pd
from io import BytesIO
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [67]:
jan_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet'
feb_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet'

In [ ]:
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet

In [76]:
df_jan = pd.read_parquet(BytesIO(requests.get(jan_url, allow_redirects=True).content))
df_feb = pd.read_parquet(BytesIO(requests.get(feb_url, allow_redirects=True).content))

In [77]:
df_jan.columns.size

19

In [79]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [80]:
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds()/60
df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds()/60
df_jan.query('duration >= 1 and duration <= 60').shape[0]/df_jan.shape[0]

0.9827547930522406

In [97]:
categorical = ['PULocationID', 'DOLocationID']
             

In [98]:
def read_data(df):
    df[categorical] = df[categorical].astype('str')
    train_dict = df[categorical].to_dict(orient='records')
    Y_train = df['duration']
    return train_dict, Y_train

In [99]:
X_train, Y_train = read_data(df_jan)
X_val, Y_val = read_data(df_feb)

In [100]:
dv = DictVectorizer()
X_train = dv.fit_transform(X_train)
X_val = dv.transform(X_val)

In [101]:
X_train.shape

(2463931, 518)

In [102]:
lr = LinearRegression()
lr.fit(X_train, Y_train)

LinearRegression()

In [103]:
y_pred = lr.predict(X_val)

In [104]:
mean_squared_error(Y_train, lr.predict(X_train), squared=False)

46.01430975765687

In [105]:
mean_squared_error(Y_val, y_pred, squared=False)

46.79235613633942

In [95]:
y_pred

array([26.37761674, 25.17854557, 31.69769092, ..., 13.23798395,
       17.7779205 , 10.73593205])